## Pregunta 3

Primero, comenzamos definiendo la estructura de un nodo de un MerkleTree

In [50]:
from __future__ import annotations # esta librería es nativa en python, por eso no está en requirements.txt
# sirve sólo para tipar de manera recursiva en una clase

class Node:
    def __init__(
        self,
        value: str,
        left_node: Node = None,
        right_node: Node = None,
        parent: Node = None
    ) -> Node:
        self.value = value
        self.left_node = left_node
        self.right_node = right_node
        self.parent = parent
    
    def shallow_copy(self): # función que retorna una copia del nodo
        node = Node(self.value)
        return node
    
    def get_proof_for(self, item: str) -> [(str, "d"|"i")]:
        if self.value == item:
            return []
        
        if self.left_node is None: # esto implica que no hay más hijos, por ende no encontramos el nodo
            return None
    
        left_node_proof = self.left_node.get_proof_for(item)
        if left_node_proof is not None: # encontramos al nodo!
            current_level_proof = (self.right_node.value, "d") 
            left_node_proof.append(current_level_proof) # agregamos este nodo actual al path hecho
            return left_node_proof # retornamos el path
        
        right_node_proof = self.right_node.get_proof_for(item)
        if right_node_proof is not None:
            current_level_proof = (self.left_node.value, "i")
            right_node_proof.append(current_level_proof) # lo mismo que la izquierda
            return right_node_proof
        
        return None # si no encontramos ni en el hijo izq, ni en el der, entonces no lo encontramos
        


Con esto, creamos la clase MerkleTree, compuesta por nodos.

In [42]:
class MerkleTree:
    def __init__(self, strings: [str], hash_func: Callable[[str], str]) -> MerkleTree:
        self.hash = hash_func
        self.__build_tree(strings)
        
    def get_root(self) -> Node:
        return self.root
    
    def get_proof_for(self, item: str) -> [(str, "d"|"i")]:
        return self.root.get_proof_for(item) # pedimos al nodo de raíz que busque al nodo
    
    def __build_tree(self, strings):
        nodes = self.__build_nodes(strings) # creamos las instancias de los nodos
        while len(nodes) != 1: # si hay ún sólo nodo, entonces es la raíz y nada más que hacer
            upper_level_nodes = []
            
            for i in range(0, len(nodes), 2):# vamos de a pares de nodos
                left_node = nodes[i]
                
                try:
                    right_node = nodes[i + 1]
                except Exception: # prbablemente sería mejor rescatar el error específico
                    right_node = nodes[i].shallow_copy() # duplicamos el nodo si es que es necesario
                    
                value = self.hash(left_node.value + right_node.value) # calculamos el hash de este nodo
                node = Node(value, left_node=left_node, right_node=right_node) # creamos el nodo actual
                left_node.parent = node # seteamos el padre de los hijos
                right_node.parent = node
                upper_level_nodes.append(node) # agregamos el nodo al nivel superior
            nodes = upper_level_nodes
        
        self.root = nodes[0] # el último nodo es la raíz
                   
                
        
    def __build_nodes(self, strings):
        nodes = [Node(string) for string in strings]
        return nodes
        
        

Definimos la función verify de acuredo a esta implementación

In [54]:
def verify(
    root: string,
    item: str,
    proof: [(str, "d"|"i")],
    hash_func: Callable[[str], str]
) -> bool:
    
    buffer = item # vamos calculando los hashes en el buffer
    for value, position in proof:
        if position == "i":
            buffer = hash_func(value + buffer)
        else:
            buffer = hash_func(buffer + value)
    return buffer == root # si el buffer tiene el mismo hash que la raíz, entonces se comprueba la prueba

Las siguientes líneas están con el único propósito de testear las implementaciones

In [51]:
# def identity(string: str) -> str:
#     return string

# tree = MerkleTree(['aaa', 'b', 'cc', 'd', 'a'], identity)
# tree.get_root().value

# tree.get_proof_for('d')

[('cc', 'i'), ('aaab', 'i'), ('aaaa', 'd')]

In [58]:
# def identity(string: str) -> str:
#     return string

# tree = MerkleTree(['aaa', 'b', 'cc', 'd', 'a'], identity)
# tree.get_root().value

# proof = tree.get_proof_for('d')

# verify(tree.get_root().value, 'e', proof, tree.hash)

False